In [1]:
%load_ext dotenv

In [2]:
%dotenv

In [3]:
import os

api_key = os.getenv("api_key")
api_key_secret = os.getenv("api_key_secret")
access_token = os.getenv("access_token")
access_token_secret = os.getenv("access_token_secret")

In [4]:
import tweepy
import re
import pandas as pd
import string
import nltk
import random
from nltk.tokenize import word_tokenize

In [5]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\dell\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [6]:
auth = tweepy.OAuthHandler(api_key, api_key_secret)
auth.set_access_token(access_token, access_token_secret)

In [7]:
api = tweepy.API(auth)

In [8]:
cursor = tweepy.Cursor(api.user_timeline, id='MichaelVaughan', tweet_mode='extended').items(10000)

In [9]:
#saving these texts
texts = []
for i in cursor:
    line = i.full_text
    line = line.strip()
    texts.append(re.sub('\n', '', line))

In [10]:
print(texts)

['Cannot wait .. 👍👍 https://t.co/hIfNGXx5fs', 'Really enjoyed watching Mo Ali play Test cricket.Brilliant at times mixed in with some entertaining stuff,he will be missed as I believe he is a great team man but it’s inevitable in this era for players who are high class at white ball to have a few years playing for Franchises', 'Jason Roy 👁👁 #IPL2021', 'Come on Europe .. Get them nipping .. #RyderCup', 'Fantastic win .. Well done 👍 https://t.co/2MC0E5Wr05', 'Never disappoints !! @imjadeja 👍👍👍 #IPL2021', 'Go on @ShaneLowryGolf !! 👍👍👍 https://t.co/kg7Lnu0VYr', 'Only way to spend a Sunday morning .. Queuing up for 30 mins to get my £30 limit of Diesel !!! #OnOn', 'Why I hope all England’s players choose the Ashes this winter https://t.co/SZmRHDhC2Q #Ashes @TelegraphSport', 'Lovely day for a game of cricket at Lords !!! 👍👍👍', 'RT @cricbuzz: ‘Greatest decision to make #MSDhoni #TeamIndia’s mentor’Watch @bhogleharsha &amp; @MichaelVaughan hail MS Dhoni the master tact…', 'RT @bhogleharsha: We

In [ ]:
# hi = 0
# en = 0

In [ ]:
# for i in texts:
#     blob = TextBlob(i)
#     if blob.detect_language() == 'en':
#         en += 1
#     elif blob.detect_language() == 'hi':
#         hi += 1

In [11]:
def clean_txt(txt):
    cleaned_txt = []
    url = re.compile(r'https?://\S+|www\.S+')
    html = re.compile(r'<.*?>')
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    punct_table = str.maketrans('', '', string.punctuation)
    for line in txt:
        line = line.lower()
        line = url.sub(r'', line)
        line = html.sub(r'', line)
        line = emoji_pattern.sub(r'', line)
        line = line.translate(punct_table)
        if len(line) >= 3:
            tokens = word_tokenize(line)
            words = [word for word in tokens if word.isalpha()]
            cleaned_txt += words
    return cleaned_txt

In [12]:
txt = clean_txt(texts)

In [13]:
txt

['can',
 'not',
 'wait',
 'really',
 'enjoyed',
 'watching',
 'mo',
 'ali',
 'play',
 'test',
 'cricketbrilliant',
 'at',
 'times',
 'mixed',
 'in',
 'with',
 'some',
 'entertaining',
 'stuffhe',
 'will',
 'be',
 'missed',
 'as',
 'i',
 'believe',
 'he',
 'is',
 'a',
 'great',
 'team',
 'man',
 'but',
 'it',
 's',
 'inevitable',
 'in',
 'this',
 'era',
 'for',
 'players',
 'who',
 'are',
 'high',
 'class',
 'at',
 'white',
 'ball',
 'to',
 'have',
 'a',
 'few',
 'years',
 'playing',
 'for',
 'franchises',
 'jason',
 'roy',
 'come',
 'on',
 'europe',
 'get',
 'them',
 'nipping',
 'rydercup',
 'fantastic',
 'win',
 'well',
 'done',
 'never',
 'disappoints',
 'imjadeja',
 'go',
 'on',
 'shanelowrygolf',
 'only',
 'way',
 'to',
 'spend',
 'a',
 'sunday',
 'morning',
 'queuing',
 'up',
 'for',
 'mins',
 'to',
 'get',
 'my',
 'limit',
 'of',
 'diesel',
 'onon',
 'why',
 'i',
 'hope',
 'all',
 'england',
 's',
 'players',
 'choose',
 'the',
 'ashes',
 'this',
 'winter',
 'ashes',
 'telegraphs

In [14]:
def create_markov_dict(txt, n_gram=2):
    markov_dict = {}
    for i in range(len(txt) - n_gram - 1):
        curr_state, next_state = "", ""
        for j in range(n_gram):
            curr_state += txt[i + j] + " "
            next_state += txt[i + j + n_gram] + " "
        curr_state = curr_state.strip()
        next_state = next_state.strip()
        if curr_state not in markov_dict:
            markov_dict[curr_state] = {}
            markov_dict[curr_state][next_state] = 1
        else:
            if next_state in markov_dict[curr_state]:
                markov_dict[curr_state][next_state] += 1
            else:
                markov_dict[curr_state][next_state] = 1
    
    for curr_state, transition in markov_dict.items():
        total = sum(transition.values())
        for state, count in transition.items():
            markov_dict[curr_state][state] = count/total
    
    return markov_dict
        
def generate_tweet(markov_dict, begin_word, length=100):
    n = 0
    curr_state = begin_word
    next_state = None
    generated_tweet = ""
    generated_tweet += curr_state + " "
    while n < length:
        next_state = random.choices(
            list(markov_dict[curr_state].keys()), 
            list(markov_dict[curr_state].values())
        )
        curr_state = next_state[0]
        generated_tweet += curr_state + " "
        n += 1
    return generated_tweet

In [15]:
model = create_markov_dict(txt, 2)
print(model.keys())

dict_keys(['can not', 'not wait', 'wait really', 'really enjoyed', 'enjoyed watching', 'watching mo', 'mo ali', 'ali play', 'play test', 'test cricketbrilliant', 'cricketbrilliant at', 'at times', 'times mixed', 'mixed in', 'in with', 'with some', 'some entertaining', 'entertaining stuffhe', 'stuffhe will', 'will be', 'be missed', 'missed as', 'as i', 'i believe', 'believe he', 'he is', 'is a', 'a great', 'great team', 'team man', 'man but', 'but it', 'it s', 's inevitable', 'inevitable in', 'in this', 'this era', 'era for', 'for players', 'players who', 'who are', 'are high', 'high class', 'class at', 'at white', 'white ball', 'ball to', 'to have', 'have a', 'a few', 'few years', 'years playing', 'playing for', 'for franchises', 'franchises jason', 'jason roy', 'roy come', 'come on', 'on europe', 'europe get', 'get them', 'them nipping', 'nipping rydercup', 'rydercup fantastic', 'fantastic win', 'win well', 'well done', 'done never', 'never disappoints', 'disappoints imjadeja', 'imjad

In [17]:
print(generate_tweet(markov_dict = model, begin_word = "going forward", length = 15))

going forward for cricketers unless they change more toursevents will either be cancelled or played with some of the best prep leading into the uk surely tells us so much natural external 
